## Minimax algorithm for the game of tic-tac-toe

In [22]:
class TicTacToe:
    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'

    def evaluate_board(self, board, player):
        # Check rows
        for i in range(0, 9, 3):
            if board[i] == board[i+1] == board[i+2] == player:
                return 10
            elif board[i] == board[i+1] == board[i+2] == 'O' if player == 'X' else 'X':
                return -10

        # Check columns
        for i in range(3):
            if board[i] == board[i+3] == board[i+6] == player:
                return 10
            elif board[i] == board[i+3] == board[i+6] == 'O' if player == 'X' else 'X':
                return -10

        # Check diagonals
        if board[0] == board[4] == board[8] == player:
            return 10
        elif board[0] == board[4] == board[8] == 'O' if player == 'X' else 'X':
            return -10
        if board[2] == board[4] == board[6] == player:
            return 10
        elif board[2] == board[4] == board[6] == 'O' if player == 'X' else 'X':
            return -10

        # If no winner, return 0
        return 0

    def minimax(self, board, depth, is_maximizing_player, player):
        score = self.evaluate_board(board, player)

        # If the game is over, return the score
        if score == 10:
            return score
        if score == -10:
            return score
        if ' ' not in board:
            return 0

        # If it's the maximizing player's turn
        if is_maximizing_player:
            best_score = float('-inf')
            for i in range(len(board)):
                if board[i] == ' ':
                    board[i] = player
                    score = self.minimax(board, depth + 1, False, 'O' if player == 'X' else 'X')
                    board[i] = ' '
                    best_score = max(best_score, score)
            return best_score

        # If it's the minimizing player's turn
        else:
            best_score = float('inf')
            for i in range(len(board)):
                if board[i] == ' ':
                    board[i] = 'O' if player == 'X' else 'X'
                    score = self.minimax(board, depth + 1, True, player)
                    board[i] = ' '
                    best_score = min(best_score, score)
            return best_score

    def get_best_move(self, board, player):
        """
        Returns the best move for the given player.
        """
        best_score = float('-inf')
        best_move = -1
        for i in range(len(board)):
            if board[i] == ' ':
                board[i] = player
                score = self.minimax(board, 0, False, 'O' if player == 'X' else 'X')
                board[i] = ' '
                if score > best_score:
                    best_score = score
                    best_move = i
        return best_move

    def print_board(self, board):
        """
        Prints the current state of the tic-tac-toe board.
        """
        print("-------------")
        for i in range(0, 9, 3):
            print("| " + board[i] + " | " + board[i+1] + " | " + board[i+2] + " |")
            print("-------------")

In [23]:
game = TicTacToe()

while True:
    # Player's turn
    game.print_board(game.board)
    print("Your turn.")
    move = int(input("Enter a number (0-8) to make your move: "))
    if game.board[move] != ' ':
        print("That spot is already taken. Try again.")
        continue
    game.board[move] = game.player

    # Check for a winner
    if game.evaluate_board(game.board, game.player) == 10:
        game.print_board(game.board)
        print(f"{game.player} wins!")
        break
    if ' ' not in game.board:
        game.print_board(game.board)
        print("It's a tie!")
        break

    # Computer's turn
    game.print_board(game.board)
    print("Computer's turn.")
    best_move = game.get_best_move(game.board, 'O')
    game.board[best_move] = 'O'

    # Check for a winner
    if game.evaluate_board(game.board, 'O') == 10:
        game.print_board(game.board)
        print("The computer wins!")
        break

-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
Your turn.
Enter a number (0-8) to make your move: 4
-------------
|   |   |   |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
Computer's turn.
-------------
| O |   |   |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
Your turn.
Enter a number (0-8) to make your move: 2
-------------
| O |   | X |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
Computer's turn.
-------------
| O | O | X |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
Your turn.
Enter a number (0-8) to make your move: 6
-------------
| O | O | X |
-------------
|   | X |   |
-------------
| X |   |   |
-------------
X wins!


##*Extend* the minimax algorithm to handle heuristic evaluation functions

---



In [19]:
class TicTacToeGame:
    def __init__(self):
        self.board = [' '] * 9
        self.current_player = 'X'

    def get_valid_moves(self, board):
        return [i for i in range(9) if board[i] == ' ']

    def is_terminal_node(self, board):
        # Check rows
        for i in range(0, 9, 3):
            if board[i] == board[i+1] == board[i+2] != ' ':
                return True
        # Check columns
        for i in range(3):
            if board[i] == board[i+3] == board[i+6] != ' ':
                return True
        # Check diagonals
        if board[0] == board[4] == board[8] != ' ':
            return True
        if board[2] == board[4] == board[6] != ' ':
            return True
        # Check if the board is full
        if ' ' not in board:
            return True
        return False

    def evaluate_board(self, board):
        player = self.current_player
        opponent = 'O' if player == 'X' else 'X'

        # Count the number of rows, columns, and diagonals won by each player
        player_rows, player_cols, player_diags = self.count_wins(board, player)
        opponent_rows, opponent_cols, opponent_diags = self.count_wins(board, opponent)

        # Calculate the score based on the number of wins and the number of empty squares
        score = (player_rows + player_cols + player_diags) * 10
        score -= (opponent_rows + opponent_cols + opponent_diags) * 10
        score += board.count(' ')

        return score

    def count_wins(self, board, player):
        rows = 0
        cols = 0
        diags = 0

        # Count rows
        for i in range(0, 9, 3):
            if board[i] == board[i+1] == board[i+2] == player:
                rows += 1

        # Count columns
        for i in range(3):
            if board[i] == board[i+3] == board[i+6] == player:
                cols += 1

        # Count diagonals
        if board[0] == board[4] == board[8] == player:
            diags += 1
        if board[2] == board[4] == board[6] == player:
            diags += 1

        return rows, cols, diags

class MinimaxWithHeuristic:
    def __init__(self, game):
        self.game = game

    def minimax(self, node, depth, maximizing_player):
        if depth == 0 or self.game.is_terminal_node(node):
            return self.game.evaluate_board(node)

        if maximizing_player:
            max_eval = float('-inf')
            for child in self.game.get_valid_moves(node):
                new_board = node.copy()
                new_board[child] = self.game.current_player
                eval = self.minimax(new_board, depth - 1, False)
                max_eval = max(max_eval, eval)
                print(f"Depth: {depth}, Move: {child}, Score: {eval}")
            return max_eval
        else:
            min_eval = float('inf')
            for child in self.game.get_valid_moves(node):
                new_board = node.copy()
                new_board[child] = 'O'
                eval = self.minimax(new_board, depth - 1, True)
                min_eval = min(min_eval, eval)
                print(f"Depth: {depth}, Move: {child}, Score: {eval}")
            return min_eval

    def get_best_move(self, node, depth):
        best_score = float('-inf')
        best_move = None

        for child in self.game.get_valid_moves(node):
            new_board = node.copy()
            new_board[child] = self.game.current_player
            score = self.minimax(new_board, depth - 1, False)
            if score > best_score:
                best_score = score
                best_move = child

        return best_move

# Example usage
game = TicTacToeGame()
minimax_with_heuristic = MinimaxWithHeuristic(game)
best_move = minimax_with_heuristic.get_best_move(game.board, 3)
print(f"Best move: {best_move}")

Depth: 1, Move: 2, Score: 6
Depth: 1, Move: 3, Score: 6
Depth: 1, Move: 4, Score: 6
Depth: 1, Move: 5, Score: 6
Depth: 1, Move: 6, Score: 6
Depth: 1, Move: 7, Score: 6
Depth: 1, Move: 8, Score: 6
Depth: 2, Move: 1, Score: 6
Depth: 1, Move: 1, Score: 6
Depth: 1, Move: 3, Score: 6
Depth: 1, Move: 4, Score: 6
Depth: 1, Move: 5, Score: 6
Depth: 1, Move: 6, Score: 6
Depth: 1, Move: 7, Score: 6
Depth: 1, Move: 8, Score: 6
Depth: 2, Move: 2, Score: 6
Depth: 1, Move: 1, Score: 6
Depth: 1, Move: 2, Score: 6
Depth: 1, Move: 4, Score: 6
Depth: 1, Move: 5, Score: 6
Depth: 1, Move: 6, Score: 6
Depth: 1, Move: 7, Score: 6
Depth: 1, Move: 8, Score: 6
Depth: 2, Move: 3, Score: 6
Depth: 1, Move: 1, Score: 6
Depth: 1, Move: 2, Score: 6
Depth: 1, Move: 3, Score: 6
Depth: 1, Move: 5, Score: 6
Depth: 1, Move: 6, Score: 6
Depth: 1, Move: 7, Score: 6
Depth: 1, Move: 8, Score: 6
Depth: 2, Move: 4, Score: 6
Depth: 1, Move: 1, Score: 6
Depth: 1, Move: 2, Score: 6
Depth: 1, Move: 3, Score: 6
Depth: 1, Move: 4, S